# Doc2VecでWikipedia全文を使用

Wiki全文（か、もしくは一部）をDoc2Vecのインプットに指定して動作確認します。

今回は２つの手法を比較しました。

- PV-DBOW：書きかけエントリーが大量に混在していたため、あまり良好な動きではありませんでした。（書きかけエントリーを除外して学習させれば、それなりの性能が上がるのかもしれません）


- PV-DM：全般的にNG

## (1) Wikipediaコンテンツファイルから本文を抽出

### (1-1) Wikipediaコンテンツ取得

Wikipedia コンテンツの一覧を、以下のURLで参照します。

https://dumps.wikimedia.org/jawiki/latest/

このページのリストから、以下のファイルのリンクをクリックしてダウンロードします。

2.4GBほどになります。

```
jawiki-latest-pages-articles.xml.bz2    21-May-2017 01:07    2492605537
```

### (1-2) Wikipedia Extractor（本文抽出ツール）の取得

下記サイト内のDownloadsからWikipedia ExtractorのPythonスクリプトを直接ダウンロードします。

http://medialab.di.unipi.it/wiki/Wikipedia_Extractor

### (1-3) Wikipediaコンテンツファイルから本文を抽出

下記コマンドで抽出を実行します。

python3 WikiExtractor.py -b 10M -o extracted jawiki-latest-pages-articles.xml.bz2

これで、extractedディレクトリ下に、10MB毎に区切られた抽出後ファイルが生成されていきます。

extractedディレクトリ配下には、AA、AB、AC・・・といったサブディレクトリが生成されます。

サブディレクトリごとに、１００本のファイルが生成されます。

（すなわちサブディレクトリごとに1GBまで）

実行には４０分ほどかかるとのことでしたが、実績としては、5/22 13:51開始〜14:26終了、なので３５分ほどで完了しています。

実行例：
```
MacBookPro-makmorit-jp:Wikipedia makmorit$ pwd
/Users/makmorit/Documents/Development/Wikipedia
MacBookPro-makmorit-jp:Wikipedia makmorit$ ls -al
total 4868576
drwxr-xr-x   4 makmorit  staff         136 May 22 13:44 .
drwxr-xr-x  10 makmorit  staff         340 May 22 13:43 ..
-rw-r--r--@  1 makmorit  staff       98379 Mar 23  2016 WikiExtractor.py
-rw-r--r--@  1 makmorit  staff  2492605537 May 22 13:27 jawiki-latest-pages-articles.xml.bz2
MacBookPro-makmorit-jp:Wikipedia makmorit$ python WikiExtractor.py -b 10M -o extracted jawiki-latest-pages-articles.xml.bz2
INFO: Loaded 0 templates in 0.0s
INFO: Starting page extraction from jawiki-latest-pages-articles.xml.bz2.
INFO: Using 7 extract processes.
WARNING: Template errors in article '自然数' (4671): title(9) recursion(0, 0, 0)
WARNING: Template errors in article '階乗' (12763): title(1) recursion(0, 0, 0)
INFO: Extracted 10000 articles (225.3 art/s)
INFO: Extracted 20000 articles (263.3 art/s)
（中略）
WARNING: Template errors in article 'プロ経営者' (3516673): title(2) recursion(0, 0, 0)
INFO: Extracted 1410000 articles (990.3 art/s)
WARNING: Template errors in article '2017年のWTAツアー' (3535919): title(2) recursion(0, 0, 0)
INFO: Extracted 1420000 articles (995.9 art/s)
INFO: Extracted 1430000 articles (910.8 art/s)
WARNING: Template errors in article 'Wikipedia:削除依頼/性液' (3583405): title(1) recursion(0, 0, 0)
INFO: Extracted 1440000 articles (986.9 art/s)
INFO: Finished 7-process extraction of 1446777 articles in 2059.6s (702.5 art/s)
MacBookPro-makmorit-jp:Wikipedia makmorit$ date
Mon May 22 14:25:52 JST 2017
MacBookPro-makmorit-jp:Wikipedia makmorit$ 
```

## (2) 抽出コンテンツからTaggedDocumentを生成

### (2-1) 抽出コンテンツの内容についての注意

下記のように、タイトルに : 文字が含まれている場合は有効な記事でないようです。

```
<doc id="1" url="https://ja.wikipedia.org/wiki?curid=1" title="Wikipedia:アップロードログ 2004年4月">
Wikipedia:アップロードログ 2004年4月
<ul>
</doc>
```

したがってTaggedDocumentへの変換時は、これらのエントリーは除外する必要があります。

### (2-2) 抽出コンテンツをTaggedDocumentへ変換


有効なドキュメントは下記のようになっています。

```
<doc id="2466881" url="https://ja.wikipedia.org/wiki?curid=2466881" title="ヒントン駅">
ヒントン駅
ヒントン駅（ヒントンえき、英語：Hinton Station）は、ウェストバージニア州 メープル・アベニューのセカンド・アヴェニュー100にある駅。 昔のチェサピーク・アンド・オハイオ鉄道の駅である。
アムトラックの停車する列車は下記の通り。
</doc>
```

これを、以下のようにして、TaggedDocumentに変換します。

分ち書きにはプロダクションのNlangクラスを使用します（すなわち品詞を一部落とします）。

In [2]:
'''
    環境準備
'''
import sys
import os
import re
learning_dir = os.path.abspath("../../") #<--- donusagi-bot/learning
os.chdir(learning_dir)
if learning_dir not in sys.path:
    sys.path.append(learning_dir)

In [3]:
def process_start_tag(line):
    '''
        docタグ開始と判定した場合
        docタグから取得したid, titleを戻す。

        ただし、titleに':' '曖昧さ回避'が
        含まれている場合は無効エントリーとし、
        docタグ開始と判定しない
    '''
    tag_start_pattern = r'<doc id="([0-9]+)" url=".+" title="([^:]+)">'

    sm = re.match(tag_start_pattern, line)
    if sm is None:
        return None, None
        
    try:
        id = sm.group(1)
        title = sm.group(2)
        
        if title.find('曖昧さ回避') > 0:
            id = None
            title = None

    except:
        id = None
        title = None
        pass

    return id, title

In [4]:
def process_end_tag(line):
    '''
        docタグの終了判定
    '''
    tag_end_pattern = r'</doc>'

    em = re.match(tag_end_pattern, line)
    if em is not None:
        return True

    return False

In [5]:
def parse_wiki_xml(lines):
    '''
        Wikipediaから抽出されたコンテンツをリスト化する。
        
        Wikipediaから抽出されたコンテンツはXML形式になっているが、
        通常Pythonで用意されているパーサーでは処理不可能なので、
        この関数でparseを行う
    '''
    parsed_wiki = []
    tag_started = False

    for line in lines:
        line = line.replace('\n', '')

        if not tag_started:
            '''
                doc タグの開始判定
            '''
            id, title = process_start_tag(line)
            if id is not None:
                tag_started = True
                first_line = True
                content = ''
            else:
                tag_started = False
            continue

        if not tag_started:
            continue

        if process_end_tag(line):
            '''
                doc タグが終了したら、
                id, title, 本文をリストに出力
            '''
            parsed_wiki.append((id, title, content))
            tag_started = False
            content = ''
            continue

        '''
            doc タグの途中なら本文として扱う
            ただし先頭行はタイトルなので
            本文として扱わない
        '''
        if first_line:
            first_line = False
        else:
            content += line
    
    return parsed_wiki

In [6]:
def parse_wiki_xml_file(file_path):
    parsed_wiki = None

    with open(file_path) as wikifile:
        lines = wikifile.readlines()
        wikifile.close()
        parsed_wiki = parse_wiki_xml(lines)

    return parsed_wiki

In [7]:
from gensim.models.doc2vec import TaggedDocument
from learning.core.nlang import Nlang

def get_tagged_document_list(wiki_extracted_file_path):
    '''
        コンテンツファイルから逐一読み込み、
        タグとコーパスをTaggedDocumentに設定
    '''
    tagged_document_list = []

    parsed_wiki = parse_wiki_xml_file(file_path)
    for wiki in parsed_wiki:
        # これは <doc> タグのidです
        tag = int(wiki[0])

        # これは <doc> タグの中身です
        sentences = wiki[2]

        separated_sentences = Nlang.split(sentences)
        words = separated_sentences.split()
        tagged_document_list.append(TaggedDocument(words=words, tags=[tag]))
        
    return tagged_document_list

In [8]:
'''
    ファイルからレコードを取得
'''
extracted_dir_path = '/Users/makmorit/Documents/Development/Wikipedia/extracted'

prefix = 'AA'
file_no = 0
file_path = os.path.join(extracted_dir_path, prefix, 'wiki_%02d' % file_no)

In [9]:
tagged_document_list = get_tagged_document_list(file_path)
tagged_document_list[0]

TaggedDocument(words=['アンパサンド', '（，', '＆）', '意味', 'する', '記号', '英語', '相当', 'する', 'ラテン語', '合', '字', '（', 'ｅｔ', 'ｃｅｔｅｒａ', '＝', 'ａｎｄ', 'ｓｏ', 'ｆｏｒｔｈ', '）', '記述', 'する', 'Ｔｒｅｂｕｃｈｅｔ', 'ＭＳ', 'フォント', '表示', 'する', 'れる', '”', 'ｅｔ', '”', '合', '字', '容易', 'わかる', '使用', '１', '世紀', '遡る', 'できる', '（', '１', '）、', '５', '世紀', '中葉', '（', '２', '，', '３', '）', '現代', '（', '４', '−', '６', '）', '至る', '変遷', 'わかる', 'Ｚ', '続く', 'ラテン', '文字', 'アルファベット', '２７', '字', '目', 'する', 'れる', '時期', 'アンパサンド', '役割', '果たす', '文字', 'ｅｔ', '呼ぶ', 'れる', '数字', '７', '似る', '記号', '（，', 'Ｕ', '＋', '２０４', 'Ａ', '）。', '記号', '現在', 'ゲール', '文字', '使う', 'れる', '記号', '名', 'アンパサンド', 'ラテン語', 'まじる', '英語', '「＆', 'それ', '自身', '”', 'ａｎｄ', '”', '表す', '（＆', 'ｐｅｒ', 'ｓｅ', 'ａｎｄ', '）', 'くずれる', '形', '英語', '言語', '名称', '多様', '日常', '的', '手書き', '場合', '欧米', 'アンパサンド', '縦', '線', '引く', '単純', '化', 'する', 'れる', '使う', 'れる', '同様', 'ｔ', '「＋（', 'プラス', '輪', '重ねる', '無声', '歯茎', '側面', '摩擦音', '示す', '発音', '記号', '使う', 'れる', 'プログラミング', '言語', 'Ｃ', '多数', '言語', 'ＡＮＤ', '演算', '子', '用いる', 'られる', 'Ｃ', 

### (2-3) TaggedDocumentをインプットとして学習

今回はマイオペのテストデータは入れないでテストします。

概ね下記のようなコードを実行して学習します。

ただし分類数がわからないので、適当に500程度のサイズに設定しておきます。

繰り返し回数はサイズの１０倍程度を指定します。

In [10]:
from gensim.models.doc2vec import Doc2Vec

def doc2vec_model_path(dm):
    model_path = 'prototype/better_algorithm/doc2vec.wikipedia.PV%d.model' % dm
    return model_path

def train(tagged_document_list, dm=0):
    '''
        tagged_document_listは、
        すべてのWikipediaのタグ付きコーパス
        (前述のtagged_document)が収容されたリスト。

        これを引数にして、
        ボキャブラリ生成／学習実行を実行します。
        
        dm=0でPV-DBOW、dm=1でPV-DMにより実行します。
        （今回は比較のため、両方で試す）
    '''
    model = Doc2Vec(dm=dm, size=500, min_count=1, iter=5000)
    model.build_vocab(tagged_document_list)
    ret = model.train(tagged_document_list)

    # 学習モデルは、ファイルに保存しておく
    model.save(doc2vec_model_path(dm))
    print('train_by_doc2vec: document vector size=%d, return=%d' % (len(model.docvecs), ret))

    return ret

In [10]:
'''
    tagged_document_listを
    使用し、学習実行（PV-DBOW）
'''
train(tagged_document_list) 

train_by_doc2vec: document vector size=1696, return=4744510453


4744510453

In [11]:
'''
    tagged_document_listを
    使用し、学習実行（PV-DM）
'''
train(tagged_document_list, dm=1) 

train_by_doc2vec: document vector size=1696, return=4744473928


4744473928

## (3) 予測実行

作成されたモデルを使用して予測を実行する関数になります。

In [12]:
from gensim import models

def predict(word, dm=0):
    '''
        予測処理にかけるコーパスを生成
        （学習セット作成時と同じ関数を使用）
    '''
    corpus = Nlang.split(word).split()

    '''
        コーパスからベクトルを生成し、
        ロードしたモデルから類似ベクトルを検索
    '''
    loaded_model = models.Doc2Vec.load(doc2vec_model_path(dm))
    inferred_vector = loaded_model.infer_vector(corpus)
    ret = loaded_model.docvecs.most_similar([inferred_vector])

    return corpus, ret

### (3-1) 適当な質問文で予測（PV-DBOW)

In [17]:
'''
    単語数がやや多い質問文を使い、質問をしてみます。
'''
predict('風邪を引いた時の治療薬を教えてください')

(['風邪', '引く', '治療', '薬', '教える'],
 [(908, 0.35965487360954285),
  (280, 0.3451248109340668),
  (329, 0.34359341859817505),
  (912, 0.3259981572628021),
  (105, 0.3258604407310486),
  (655, 0.3153131902217865),
  (256, 0.3124464750289917),
  (780, 0.3012807071208954),
  (1460, 0.30003029108047485),
  (853, 0.2989971339702606)])

元ねたと答え合わせ。

２件ほど本文にキーワードがあるもの（id=329と、id=655）が引っかかっている様です。

そのほかは、書きかけエントリーに引っかかってしまいました。
```
<doc id="908" url="https://ja.wikipedia.org/wiki?curid=908" title="高橋幸二">　<---書きかけエントリー
<doc id="280" url="https://ja.wikipedia.org/wiki?curid=280" title="ゴスペル">　<---書きかけエントリー

<doc id="329" url="https://ja.wikipedia.org/wiki?curid=329" title="風邪">
風邪（かぜ、common cold, nasopharyngitis, rhinopharyngitis, acute coryza, a cold）とは、ウイルスによる上気道感染症であり（以下略）

<doc id="105" url="https://ja.wikipedia.org/wiki?curid=105" title="タイ">　<---書きかけエントリー
<doc id="912" url="https://ja.wikipedia.org/wiki?curid=912" title="高橋陽子">　<---書きかけエントリー

<doc id="655" url="https://ja.wikipedia.org/wiki?curid=655" title="水木しげる">
（中略）帰還してまもなく行軍中に風邪を引いた際にマラリアを発症（以下略）

<doc id="256" url="https://ja.wikipedia.org/wiki?curid=256" title="サンバ">　<---書きかけエントリー
<doc id="780" url="https://ja.wikipedia.org/wiki?curid=780" title="佐藤宏之">　<---書きかけエントリー
<doc id="1460" url="https://ja.wikipedia.org/wiki?curid=1460" title="FTP">　<---書きかけエントリー
<doc id="853" url="https://ja.wikipedia.org/wiki?curid=853" title="森本さやか">　<---書きかけエントリー
```

書きかけのエントリーの例

```
<doc id="861" url="https://ja.wikipedia.org/wiki?curid=861" title="JET">
JET

JET

</doc>
```


### (3-2) 適当な質問文で予測（PV-DM)

In [18]:
'''
    単語数がやや多い質問文を使い、質問をしてみます。
'''
predict('風邪を引いた時の治療薬を教えてください', dm=1)

(['風邪', '引く', '治療', '薬', '教える'],
 [(1642, 0.23558662831783295),
  (608, 0.2341582179069519),
  (1048, 0.23387373983860016),
  (1313, 0.2234296351671219),
  (246, 0.2233436405658722),
  (161, 0.22315767407417297),
  (1440, 0.22186896204948425),
  (245, 0.22185118496418),
  (820, 0.2206314206123352),
  (302, 0.22027365863323212)])

元ねたと答え合わせ。

書きかけエントリーには引っかかりませんでした。

ただし、風邪、治療、教えるの意味にかかるドキュメントは一つもありませんでした。

```
<doc id="1642" url="https://ja.wikipedia.org/wiki?curid=1642" title="ユニバーサル・シリアル・バス">
<doc id="608" url="https://ja.wikipedia.org/wiki?curid=608" title="大島司">
<doc id="1048" url="https://ja.wikipedia.org/wiki?curid=1048" title="羽海野チカ">
<doc id="1313" url="https://ja.wikipedia.org/wiki?curid=1313" title="雷句誠">
<doc id="246" url="https://ja.wikipedia.org/wiki?curid=246" title="冨樫義博">
<doc id="161" url="https://ja.wikipedia.org/wiki?curid=161" title="著作権">
<doc id="1440" url="https://ja.wikipedia.org/wiki?curid=1440" title="白倉由美">
<doc id="245" url="https://ja.wikipedia.org/wiki?curid=245" title="寺沢武一">
<doc id="820" url="https://ja.wikipedia.org/wiki?curid=820" title="閏月">
<doc id="302" url="https://ja.wikipedia.org/wiki?curid=302" title="将軍">
```

In [20]:
'''
    タイトルに近い文言で、Google検索っぽく質問をしてみます。
'''
predict('風邪', dm=1)

(['風邪'],
 [(908, 0.3303205370903015),
  (329, 0.30442649126052856),
  (346, 0.2975710332393646),
  (853, 0.2970205247402191),
  (1084, 0.2935115098953247),
  (780, 0.2880999445915222),
  (1151, 0.2864486277103424),
  (1110, 0.2839278280735016),
  (1182, 0.28222888708114624),
  (387, 0.28077220916748047)])

１件だけ、「風邪」が記述されているドキュメントが引っかかっただけでした。
```
<doc id="908" url="https://ja.wikipedia.org/wiki?curid=908" title="高橋幸二">　<---書きかけエントリー

<doc id="329" url="https://ja.wikipedia.org/wiki?curid=329" title="風邪">
風邪（かぜ、common cold, nasopharyngitis, rhinopharyngitis, acute coryza, a cold）とは、ウイルスによる上気道感染症であり（以下略）

<doc id="346" url="https://ja.wikipedia.org/wiki?curid=346" title="冨樫">　<---書きかけエントリー
<doc id="853" url="https://ja.wikipedia.org/wiki?curid=853" title="森本さやか">　<---書きかけエントリー

<doc id="1084" url="https://ja.wikipedia.org/wiki?curid=1084" title="FORTRAN">　<---「風邪」を含まない

<doc id="780" url="https://ja.wikipedia.org/wiki?curid=780" title="佐藤宏之">　<---書きかけエントリー

<doc id="1151" url="https://ja.wikipedia.org/wiki?curid=1151" title="荒俣宏">　<---「風邪」を含まない
<doc id="1110" url="https://ja.wikipedia.org/wiki?curid=1110" title="経済学">　<---「風邪」を含まない
<doc id="1182" url="https://ja.wikipedia.org/wiki?curid=1182" title="誘電体">　<---「風邪」を含まない
<doc id="387" url="https://ja.wikipedia.org/wiki?curid=387" title="岐阜県">　<---「風邪」を含まない

```